In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0`

import $ivy.`org.apache.kafka:kafka-clients:3.5.1`
import $ivy.`org.testcontainers:testcontainers:1.19.3`
import $ivy.`org.testcontainers:kafka:1.19.3`

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$

In [2]:

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._

import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val kafkaContainer = new KafkaContainer()

kafkaContainer.start()

val bootstrapServers = kafkaContainer.getBootstrapServers
val kafkaProps = new Properties()
kafkaProps.put("bootstrap.servers", bootstrapServers)
kafkaProps.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
kafkaProps.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val kafkaProducer = new KafkaProducer[String, String](kafkaProps)

def sendToKafka(topic: String, values: Seq[String]): Unit = {
 
  values.foreach { value =>
    val record = new ProducerRecord[String, String](topic, value)
    kafkaProducer.send(record)
  }
  kafkaProducer.flush()
}

def readFromKafkaSpark(topic: String): DataFrame = {
  spark.read
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrapServers)
  .option("subscribe", topic)
  .load()
  .selectExpr("CAST(value AS STRING) as message")
}

val kafkaTopic = "my-topic-scala"


1 deprecation; re-run enabling -deprecation for details, or try -help
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/30 17:34:10 WARN Utils: Your hostname, DESKTOP-CN01VS3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 17:34:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/30 17:34:10 INFO SparkContext: Running Spark version 3.5.0
25/03/30 17:34:10 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/03/30 17:34:10 INFO SparkContext: Java version 17.0.14
25/03/30 17:34:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 17:34:10 INFO ResourceUtils: ==============================================================
25/03/30 17:34:10 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/30 17:34:10 INFO ResourceUtils: ========================

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._
import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties
spark: SparkSession = org.apache.spark.sql.SparkSession@4f1d1fe3
import spark.implicits._
kafkaContainer: KafkaContainer = GenericContainer(extraHosts=[], image=RemoteDockerImage(imageName=confluentinc/cp-kafka:5.4.3, imagePullPolicy=DefaultPullPolicy(), imageNameSubstitutor=org.testcontainers.utility.ImageNameSubstitutor$LogWrappedImageNameSubstitutor@2104655a), volumesFroms=[], linkedContainers={}, startupCheckStrategy=org.testcontainers.containers.startupcheck.IsRunningStartupCheckStrategy@1e012dca, startupAttempts=1, workingDirectory=null, shmSize=null, copyToFileContainerPathMap={}, copyToTransferableContainerPathMap={}, dependencies=[], dockerClient=LazyDockerClient, containerId=1b2790461c4707b170e4ecd66efb7a75461da6e9701ffaec950969fad187f09f, containerInfo

In [3]:
val data = Seq("hi", "there", "from", "scala")

sendToKafka(kafkaTopic, data)

25/03/30 17:34:35 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 4 : {my-topic-scala=LEADER_NOT_AVAILABLE}


data: Seq[String] = List("hi", "there", "from", "scala")

In [4]:
readFromKafkaSpark(kafkaTopic).show()

25/03/30 17:34:38 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/30 17:34:38 INFO SharedState: Warehouse path is 'file:/home/eyal/code/notebooks/spark-warehouse'.
25/03/30 17:34:39 INFO CodeGenerator: Code generated in 286.722522 ms
25/03/30 17:34:39 INFO CodeGenerator: Code generated in 39.929272 ms
25/03/30 17:34:41 INFO AdminClientConfig: AdminClientConfig values: 
	auto.include.jmx.reporter = true
	bootstrap.servers = [PLAINTEXT://localhost:58094]
	client.dns.lookup = use_all_dns_ips
	client.id = 
	connections.max.idle.ms = 300000
	default.api.timeout.ms = 60000
	metadata.max.age.ms = 300000
	metric.reporters = []
	metrics.num.samples = 2
	metrics.recording.level = INFO
	metrics.sample.window.ms = 30000
	receive.buffer.bytes = 65536
	reconnect.backoff.max.ms = 1000
	reconnect.backoff.ms = 50
	request.timeout.ms = 30000
	retries = 2147483647
	retry.backoff.ms = 100
	sasl.client.callback.handler.class = null
	sasl.jaas.c

+-------+
|message|
+-------+
|     hi|
|  there|
|   from|
|  scala|
+-------+

